# <font color=blue>OmegAlpes Tutorial 3 - Heat and Electricity combined system
`Prerequisite :` <a href="./OmegAlpes_Tutorial_1.ipynb">Tutorial 1,</a>
    <a href="./OmegAlpes_Tutorial_2.ipynb">Tutorial 2</a><br><br><br>
    
<font color=red>There are some parts in this tutorial that has already been covered in previous Tutorial 1 and 2 and will not be covered here.</font>
    
    In this tutorial, we will learn to add another type of energy (thermal) into our energy model. With these feature, we will explore the suggestion to consider utilizing water heater from domestic hot water to reduce John electricity import.
    
   `Learning outcomes :`
>1. New vector in the model : thermal energy
2. Working with multiple nodes
3. Electrical to Heat conversion units
4. Processing results
   
    At the end of this tutorial, you will be able to find out whether adding thermal energy will be better outcome than the other solutions

<img src="./images/4-heat.JPG" width="550" height="150" >
    <center>Figure 1. John's house layout</center>

In [ ]:
from Python_Scripts.Omegalpes_tutorial_3 import *

    Similar to previous Tutorial 1. Try to have similar steps to have a good programming structure.

## Create time unit   

In [ ]:
time = TimeUnit(periods=24*1, dt=1)

    where :
    dt = 1 hour / 60 minutes since all of the production and consumption data is in 1-hour timestep
    period = 24 hours. You can have longer period if the data is available.
    
`to learn more in detail` : <a href="https://omegalpes.readthedocs.io/en/latest/api/general.html#module-omegalpes.general.time">time module</a>

## Create empty model
    

In [ ]:
model = OptimisationModel(time=time, name='tutorial_3')

  
`to learn more in detail` : <a href="https://omegalpes.readthedocs.io/en/latest/api/general.html#module-omegalpes.general.optimisation.model">model module</a>

## Create energy unit

### Create fixed energy units

In [ ]:
consumption_profile = open("data/house con 24.csv", "r")

house_cons_file = [p for p in map(float, consumption_profile)]

house_consumption = FixedConsumptionUnit(time, 'johns_consumption', p=house_cons_file, energy_type=elec)

In [ ]:
pv_profile = open("data/pv prod 24.csv", "r")

pv_production_daily = [p for p in map(float, pv_profile)]

pv_production = FixedProductionUnit(time, name='pv_production', p=pv_production_daily ,energy_type=elec)

    Here, we have another new fixed consumption unit, which is domestic hot water. Its usage profile is given.

In [ ]:
dhw_profile = pd.read_csv("data/DHW 24.csv", sep=';')

dhw_load = dhw_profile['kwh'].tolist()

dhw = FixedConsumptionUnit(time, name="domestic_hot_water", p=dhw_load, energy_type=thermal)

    As you can see, domestic hot water has energy type attribute of thermal
    
    Now, we have created 3 fixed energy units
<img src="./images/4-fixed.JPG" width="550" height="150">
<center>Figure 2. Adding fixed units to model</center>

### Create variable energy units

In [ ]:
grid_import = VariableProductionUnit(time, 'grid_import',energy_type=elec, p_min=0)

grid_export = VariableConsumptionUnit(time, 'grid_export', energy_type=elec, p_min=0)

wm_profile = open("data/washer 24.csv", "r")

wm_cons_file = [p for p in map(float, wm_profile)]

wm_consumption = ShiftableConsumptionUnit(time, 'washing_machine_consumption', power_values=wm_cons_file,
                                          energy_type=elec)

### Create conversion unit and storage unit
    Next, we create conversion unit, or ElectricalToThermalConversionUnit specificly to handle heat-electrical conversion.  This is a unit with an electricity consumption and a thermal production linked by and electrical to thermal ratio.
    
    In this case, this conversion unit is a water heater. Water heater converts 90% of the inlet electricity into heat. It will be modeled as an ElectricalToThermalConversionUnit with a constant conversion factor as follows:

In [ ]:
water_heater = ElectricalToThermalConversionUnit(time, name="water_heater", elec_to_therm_ratio=0.9)

    Water heater unit has attributes of :
    - time that refers to timeunit as all other energy units
    - name as result label
    - electrical to tharmal conversion ratio

`to learn more in detail` : <a href="https://omegalpes.readthedocs.io/en/latest/api/energy_package.html#module-omegalpes.energy.units.conversion_units">conversion units module</a>

    The produced energy then will be stored in water tank. So, we create storage unit for this purpose

In [ ]:
water_tank = StorageUnit(time, name="water_tank", self_disch_t=0.05, soc_min=0.2, energy_type=thermal,
                         ef_is_e0=True, capacity=6000)

    Storage unit has some new attributes : thermal energy type and defined capacity to represent the available space.
    
     Water tank has a 6 kWh capacity, which cannot be discharged below 20% of its capacity and must have the same final state of charge as its initial one. A self-discharge rate of 5% per hour is added.

    Now, we will add all these energy units to our model.
<img src="./images/4-allunit.JPG" width="650" height="150">
<center>Figure 3. Adding variable units and thermal conversion units to model</center>
<br>

## Define model objective

In [ ]:
grid_import.minimize_production()

`learn more optimization objective functions for energy units:`<a href="https://omegalpes.readthedocs.io/en/latest/api/energy_package.html#module-omegalpes.energy.units.production_units">production units module</a> & <a href="https://omegalpes.readthedocs.io/en/latest/api/energy_package.html#module-omegalpes.energy.units.consumption_units"> consumption units module</a>

## Create energy nodes
    Now we create energy nodes, here is another challenge

In [ ]:
%run Python_Scripts/quiz/quiz_tutorial41

    Although it is common practice to seperate nodes by energy type. There are also several other practices, such as seperating nodes to simulate actors objectives. You can learn further about it to improve your level in OmegAlpes!
    
    Now, Let's create the nodes!

In [ ]:
elec_node = EnergyNode(time, name="electrical_node", energy_type=elec)

elec_node.connect_units(pv_production, house_consumption, grid_import, grid_export, wm_consumption,
                    water_heater.elec_consumption_unit)

In [ ]:
heat_node = EnergyNode(time, name="heat_node", energy_type=thermal)
    
heat_node.connect_units(dhw, water_tank, water_heater.thermal_production_unit)

    Pay attention to each energy units connected to the node. Water heater as conversion unit has 2 parts which are connected to both node each. As explained before, two of these units are linked to each other to handle energy conversion. Let's observe below figure to have a better understanding of the model.
    
<img src="./images/4-node.JPG" width="650" height="150">
<center>Figure 4. Adding nodes to model</center>




`to learn more in detail` : <a href="https://omegalpes.readthedocs.io/en/latest/api/energy_package.html#module-omegalpes.energy.energy_nodes">energy nodes module</a>

## Add the energy nodes to the optimisation model
    Add all nodes to the model
    
    Remember to check that the timeunit is the same for the model and all the units

In [ ]:
model.add_nodes(elec_node, heat_node)



`to learn more in detail` : <a href="https://omegalpes.readthedocs.io/en/latest/api/general.html#module-omegalpes.general.optimisation.model">optimisation model</a>

## Launch the optimisation

In [ ]:
model.solve_and_update()



`to learn more in detail` : <a href="https://omegalpes.readthedocs.io/en/latest/api/general.html#module-omegalpes.general.optimisation.model">optimisation model</a>

## Plot result
    Plot energy flow at both nodes

In [ ]:
plot_node_energetic_flows(elec_node)
plot_node_energetic_flows(heat_node)

`learn more plotting functions:` : <a href="https://omegalpes.readthedocs.io/en/latest/api/general.html#module-omegalpes.general.utils.plots">plots module</a>

## Result processing
    *Add quiz to calculate total energy

`learn more plotting functions:` : <a href="https://omegalpes.readthedocs.io/en/latest/api/general.html#module-omegalpes.general.utils.plots">plots module</a>   


## Discussion
    What do you think about the result? What impact it does by adding heat conversion into the model? Compate the result with the other simulation you have done ! and try combining all level 2 skills you get and propose a better recommendation for John!

    *need to reduce usage profile for dhw since it is removed. 
    *realize dryer is not there? you can add
    *add final quiz
    
    Let us return to the dashboard so we can progress more in finding optimal solution for John with the other level 2 Tuto-rial. Don't forget to collect 1 level 2 magic word in this tutorial by finishing tasks in order to unlock stuffs! and the sixth and final magic word is 'OMEGALPES'
